<a href="https://colab.research.google.com/github/Abhiramiraju003/RNN/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy  scikit-learn tensorflow transformers fasttext matplotlib seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313474 sha256=4c3ddacfcd0a7c3885329dc3d5e4affe72aeb205a9567f6e33dc31f34ab2eafd
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import fasttext


In [ ]:
df=pd.read_csv('/content/1429_1.csv')

<ipython-input-4-42c3cc30ba5a>:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/1429_1.csv')


In [ ]:
import nltk # Import the nltk library
nltk.download('stopwords')
nltk.download('punkt_tab')  # Download the punkt_tab data package
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import nltk # Import the nltk library
nltk.download('stopwords')
nltk.download('punkt_tab')  # Download the punkt_tab data package
stop_words = set(nltk.corpus.stopwords.words('english'))

def preprocess_text(text):
    # Check if the text is a string before applying lowercasing
    if isinstance(text, str):
        text = text.lower()
        text = nltk.word_tokenize(text)
        text = [word for word in text if word not in stop_words]
        return ' '.join(text)
    else:
        # Handle non-string values (e.g., NaN)
        return ""  # Or any other appropriate handling
# Print the columns of your DataFrame to see the actual column names
print(df.columns)

# Assuming the actual column name containing text data is 'review_text'
df['reviews.text'] = df['reviews.text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['reviews.text'], df['reviews.rating'], test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score



# Convert reviews to string format
X = df['reviews.text'].astype(str)
y = df['reviews.rating']

# Convert ratings to binary sentiment (Negative: 1-2, Positive: 3-5)
y_binary = np.where(y < 3, 0, 1)

# **Balance Dataset (Oversampling instead of Undersampling)**
positive_indices = np.where(y_binary == 1)[0]
negative_indices = np.where(y_binary == 0)[0]

if len(positive_indices) > len(negative_indices):
    negative_indices = np.random.choice(negative_indices, size=len(positive_indices), replace=True)  # Oversample
else:
    positive_indices = np.random.choice(positive_indices, size=len(negative_indices), replace=True)

balanced_indices = np.concatenate([positive_indices, negative_indices])
np.random.shuffle(balanced_indices)

X = X.iloc[balanced_indices]
y_binary = y_binary[balanced_indices]

# **Train-Test Split**
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# **Tokenization & Padding**
max_words = 10000  # Increased vocabulary size
max_length = 200   # Sentence length

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

# **Print Tokenized Example for Debugging**
example_text = "This product is amazing!"
example_seq = tokenizer.texts_to_sequences([example_text])
print("Example tokenized review:", example_seq)

# **Improved RNN Model**
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_length),
    SimpleRNN(units=128, activation='tanh', return_sequences=True),  # More RNN layers
    Dropout(0.3),
    SimpleRNN(units=64, activation='tanh', return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# **Compile Model**
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# **Early Stopping to Avoid Overfitting**
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# **Train Model**
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test), callbacks=[early_stopping])

# **Evaluate Model**
y_pred_probs = model.predict(X_test_padded)
threshold = 0.4  # Adjusted threshold for better classification
y_pred_binary = (y_pred_probs > threshold).astype('int32')

# **Metrics**
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"\n✅ Model Evaluation:")
print(f"🔹 Accuracy: {accuracy:.4f}")
print(f"🔹 Precision: {precision:.4f}")
print(f"🔹 Recall: {recall:.4f}")
print(f"🔹 F1 Score: {f1:.4f}")
print(f"\n Confusion Matrix:\n {confusion_matrix(y_test, y_pred_binary)}")

# **User Input Prediction**
while True:
    user_input = input("\n📝 Enter a review (or type 'exit' to quit): ").strip()

    if user_input.lower() == 'exit':
        print("👋 Exiting...")
        break

    if not user_input:
        print("⚠️ Empty input! Please enter a valid review.")
        continue

    user_input_seq = tokenizer.texts_to_sequences([user_input])
    user_input_padded = pad_sequences(user_input_seq, maxlen=max_length, padding='post')

    y_pred_user = model.predict(user_input_padded)

    # **Use Adaptive Thresholding**
    sentiment = "🟢 Positive" if y_pred_user > threshold else "🔴 Negative"
    print(f"Predicted Sentiment: {sentiment} (Score: {y_pred_user[0][0]:.4f})")


Example tokenized review: [[810, 17, 6906, 301]]
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1693/1693 ━━━━━━━━━━━━━━━━━━━━ 286s 166ms/step - accuracy: 0.5025 - loss: 0.6999 - val_accuracy: 0.4948 - val_loss: 0.6945
Epoch 2/10
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 271s 160ms/step - accuracy: 0.5051 - loss: 0.6932 - val_accuracy: 0.5407 - val_loss: 0.6907
Epoch 3/10
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 320s 159ms/step - accuracy: 0.5292 - loss: 0.6916 - val_accuracy: 0.5468 - val_loss: 0.6889
Epoch 4/10
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 274s 162ms/step - accuracy: 0.5380 - loss: 0.6901 - val_accuracy: 0.5479 - val_loss: 0.6894
Epoch 5/10
1693/1693 ━━━━━━━━━━━━━━━━━━━━ 323s 163ms/step - accuracy: 0.5346 - loss: 0.6906 - val_accuracy: 0.5448 - val_loss: 0.6907
424/424 ━━━━━━━━━━━━━━━━━━━━ 15s 36ms/step

✅ Model Evaluation:
🔹 Accuracy: 0.5052
🔹 Precision: 0.5052
🔹 Recall: 1.0000
🔹 F1 Score: 0.6712

 Confusion Matrix:
 [[   0 6700]
 [   0 6840]]

📝 Enter a review (or type 'exit' to quit): i love this product
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted Sentiment: 🟢 Positive (Score: 0.5375)

📝 E